# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldest review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [68]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode
import time
import pandas as pd
import config
config.host

'flatiron.cdhanbzj0dic.us-east-2.rds.amazonaws.com'

In [70]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = 'jonandomar07',
)

In [71]:
cursor = cnx.cursor()
db_name = 'yelp4'

In [ ]:
def create_database(cursor, database): # cursor created above and database to be used
    # this creates the data base
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

# this will create a database if it does not work
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [73]:
## Create new DB
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = 'jonandomar07',
    database = 'yelp4'
)

print(cnx) # it will print a connection object if everything is find

In [74]:
cursor = cnx.cursor()

In [ ]:
def drop_everything():
    cursor.execute("""DROP TABLE businesses;""")
    cursor.execute("""DROP TABLE reviews2;""")
    cnx.commit()

In [ ]:
# Create a table for the Businesses - We did that on mysql
cursor.execute("""CREATE TABLE IF NOT EXISTS businesses 
                  (business_id VARCHAR(50) PRIMARY KEY, alias TEXT, name TEXT, 
                  review_count INTEGER, rating FLOAT, price TEXT)""")
cnx.commit()

In [ ]:
# # Create a table for the reviews - We did that on mysql
cursor.execute("""CREATE TABLE IF NOT EXISTS reviews2(
                   review_id VARCHAR(50) NOT NULL PRIMARY KEY,
                   rating TEXT,
                   time_created TEXT,
                   business_id VARCHAR(50),
                   FOREIGN KEY fk_busi(business_id)
                   REFERENCES businesses(business_id)
                   )""")

cnx.commit()

## Part 2: Create ETL pipeline for the business data from the API

In [5]:
import json
import requests

In [75]:
term = 'italian'
location = 'New York NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

In [76]:
api_key = 'apC6W5V_e5MB6oVHzSFz7CRmKSaVKBDWlNDWrIhZDwwYHOyUs5abXwKLmxSf6M1CExNI4fYPLBKq6xVhuTS0TGygAypfTT9feHgDbi4xnOY3VTb3Vt1LySr1qfY5XXYx'

In [77]:
# write a function to make a call to the API
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [78]:
data_italian = yelp_call(url_params, api_key)

In [79]:
data_italian

[{'id': 'DoSU8IPq-Py_YV3kYmXPfQ',
  'alias': 'trattoria-trecolori-new-york',
  'name': 'Trattoria Trecolori',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/G8vBgHxbEfoONO0JpLX3Qw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/trattoria-trecolori-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=txnhmZGNUmhSEfHSekziLQ',
  'review_count': 2162,
  'categories': [{'alias': 'italian', 'title': 'Italian'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.75998, 'longitude': -73.98671},
  'transactions': ['restaurant_reservation', 'pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '254 West 47th St',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'zip_code': '10036',
   'country': 'US',
   'state': 'NY',
   'display_address': ['254 West 47th St', 'New York, NY 10036']},
  'phone': '+12129974540',
  'display_phone': '(212) 997-4540',
  'distance': 6083.735166572698

In [14]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def all_results(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur # number I am at to offset the restults
        # you can parse to avoid appending
        results.extend(yelp_call(url_params, api_key))
#         results_50 = yelp_call(url_params, api_key)
#         for business in results_50:
#             dynamic_data_entry(business)
        # use dynamic insert function here
        time.sleep(1) #Wait a second
        cur += 50
    return results

In [80]:
data_italian_all = all_results(url_params, api_key)

8200 total matches found.


In [81]:
len(data_italian_all)

1000

In [82]:
data_italian_all[0]

{'id': 'DoSU8IPq-Py_YV3kYmXPfQ',
 'alias': 'trattoria-trecolori-new-york',
 'name': 'Trattoria Trecolori',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/G8vBgHxbEfoONO0JpLX3Qw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/trattoria-trecolori-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=txnhmZGNUmhSEfHSekziLQ',
 'review_count': 2162,
 'categories': [{'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.75998, 'longitude': -73.98671},
 'transactions': ['delivery', 'pickup', 'restaurant_reservation'],
 'price': '$$',
 'location': {'address1': '254 West 47th St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10036',
  'country': 'US',
  'state': 'NY',
  'display_address': ['254 West 47th St', 'New York, NY 10036']},
 'phone': '+12129974540',
 'display_phone': '(212) 997-4540',
 'distance': 6083.735166572698}

In [18]:
def dynamic_data_entry(resturant):
    business_id = resturant['id']
    alias = resturant['alias']
    name = resturant['name']
    review_count = resturant['review_count']
    rating = resturant['rating']
    if 'price' in resturant.keys():
        price = resturant['price']
    else:
        price = '' #None would have been better
    cursor = cnx.cursor()
    cursor.execute("""INSERT INTO businesses (business_id, alias, name, review_count, rating, price) 
                          VALUES (%s, %s, %s, %s, %s, %s)""", (business_id, alias, name, review_count, rating, price))
    cnx.commit()

In [19]:
for i in list(range(len(data_italian_all))):
    dynamic_data_entry(data_italian_all[i])

IntegrityError: 1062 (23000): Duplicate entry 'qhyll2gPT-gGDRP6JGn1aQ' for key 'PRIMARY'

In [20]:
cursor.execute("""SELECT * FROM businesses;""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,business_id,alias,name,review_count,rating,price
0,-0fjWWC2SY6kNxGfD8PKGg,palermo-restaurant-and-bar-astoria-2,Palermo Restaurant & Bar,188,4.5,$$
1,-58--bg-KKUNArp18U4llA,da-umberto-new-york-2,da Umberto,161,4.0,$$$
2,-5xLqs3iGFbBkJG3QNh6Bw,sacs-place-of-jackson-heights-jackson-heights,Sac's Place Of Jackson Heights,112,4.5,$
3,-7iUpGTNDdy4WjxLGYp-_g,trattoria-italienne-new-york,Trattoria Italienne,121,4.5,$$$
4,-8_vEcBA2Lqd8786S-Ss-g,novita-restaurant-new-york,Novita Restaurant,293,4.0,$$$
5,-FJ0bOC7okZbKvza8ZPzpw,ristoro-del-cinghiale-new-york,Ristoro Del Cinghiale,38,4.0,
6,-Gc5sTFxoXM3Tiv_dkSJZg,cecconis-dumbo-brooklyn,Cecconi's Dumbo,540,3.5,$$$
7,-hUvO0C0A-pGZdiSKvjoFw,babbo-new-york,Babbo,2345,4.0,$$$$
8,-KrQ1gOiyXCbffsWyc5SNQ,the-leopard-at-des-artistes-new-york,The Leopard at des Artistes,207,4.0,$$$
9,-neJTu9rpYM8-wjwA9WoIA,lex-restaurant-new-york,Lex Restaurant,88,3.5,$$


## Part 3: Create ETL pipeline for the review data from the API

In [21]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
business_ids = []
for business in data_italian_all:
    business_ids.append(business['id'])

In [23]:
len(business_ids)

1000

In [36]:
# write a function that take a business id
# and makes a call to the API for reivews
# then parse out the relevant information
def yelp_call_reviews(url_params, api_key, id_b):
    url = f'https://api.yelp.com/v3/businesses/{id_b}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    if 'reviews' in response.json().keys():
        data = response.json()['reviews']
    else:
        data = ''
    return data

In [37]:
def all_results_reviews(url_params, api_key, id_b_list):
    results = []
    for id_b in id_b_list:
        url = f'https://api.yelp.com/v3/businesses/{id_b}/reviews'
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        list_o_dic = yelp_call_reviews(url_params, api_key, id_b)
        for i in list(range(len(list_o_dic))):
            list_o_dic[i]['business_id'] = id_b
        #inesrt into db
        results.extend(list_o_dic)
        time.sleep(1) #Wait a second
    return results

In [38]:
all_reviews_italian = all_results_reviews(url_params, api_key, business_ids)

In [40]:
all_reviews_italian[:10]
# write a function to insert the parsed data into the reviews table

[{'id': 'uXCdFI1_bOcyanAMwBNfqQ',
  'url': 'https://www.yelp.com/biz/trattoria-trecolori-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&hrid=uXCdFI1_bOcyanAMwBNfqQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=txnhmZGNUmhSEfHSekziLQ',
  'text': 'I am a failure.\n\nI got to the last bite of dessert and realized, to my utter horror, that I had neglected to take pictures of anything past the...',
  'rating': 5,
  'time_created': '2019-07-12 15:12:40',
  'user': {'id': 'BHqTHehSmryTM7NZclzIpg',
   'profile_url': 'https://www.yelp.com/user_details?userid=BHqTHehSmryTM7NZclzIpg',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/jSVT5HMAAVYoKMpmKQG5rg/o.jpg',
   'name': 'Michelle V.'},
  'business_id': 'DoSU8IPq-Py_YV3kYmXPfQ'},
 {'id': 'I9itx4esmLqYYznjoiSczw',
  'url': 'https://www.yelp.com/biz/trattoria-trecolori-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&hrid=I9itx4esmLqYYznjoiSczw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source

In [41]:
len(all_reviews_italian)

2989

In [ ]:
# with open('all_reviews_italian.json', 'w') as fp:
#     json.dump(all_reviews_italian, fp)

In [86]:
f = open('all_reviews_italian.json')
reviews = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [87]:
reviews[0]

NameError: name 'reviews' is not defined

In [44]:
## Part 4: Write SQL queries that will answer the questions posed. 
# def dynamic_data_entry_reviews(review):
#     rating_id = review['id']
#     rating = review['rating']
#     time_created = review['time_created']
#     business_id = review['business_id']
    
#     cursor = cnx.cursor()
#     cursor.execute("""INSERT INTO reviews2 (review_id, rating, time_created, business_id) 
#                           VALUES (%s, %s, %s, %s)""", (rating_id, rating, time_created, business_id))
#     cnx.commit()


In [45]:
# for review in list(range(len(all_reviews_italian))):
#     dynamic_data_entry_reviews(all_reviews_italian[review])

IntegrityError: 1062 (23000): Duplicate entry '1JeqC-iF2rc5he7Yu23T6g' for key 'PRIMARY'

In [88]:
cursor.execute("""SELECT * FROM reviews2;""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,review_id,rating,time_created,business_id
0,-0-V0xoVWd1PrvA6zsxl1w,2,2019-06-02 15:58:35,wLzJbddDixAEZrF7VT6_hw
1,-09gdrip7Pa9yWoe13bN2Q,5,2019-05-29 20:04:55,xbrQ2dHuQoFJmNxAuOky4A
2,-1PO0Ca4fuFJDbESG_70Og,3,2019-06-15 13:21:16,C3E8CVVwzKtOv_sRR1WuRQ
3,-25zoB7o0DkroQ0jw-xGKg,5,2019-05-28 16:12:54,vVVmHacu2YAxKfpIjhfd1g
4,-2KGeelEKyvrQ3liocBvbg,4,2019-03-15 13:59:36,XlhWNQRGoWyybfXpGBgE3Q
5,-3qrXfxDw-lPWH5XaZ9SXw,5,2019-07-11 14:28:07,KpBHy3eOYh1OV0_6Smxs3w
6,-48ykU2TEpnZfHBkf2bPBw,5,2019-07-16 12:01:33,WG639VkTjmK5dzydd1BBJA
7,-4wXRdUZ5BLc_eRHdJBwUw,5,2019-07-09 18:27:15,VtJd1l64kqUUyWacwK0J2Q
8,-5gUbwjENJr4Qvc12WWN9w,5,2019-04-20 15:19:58,1ZKbUeFAs0ltSnAYq4ELKw
9,-5_RkfWivVmS9oGOZDVGmA,5,2019-06-23 17:03:01,nPdElJudVWoNORVxV57Umg


In [ ]:
#### SQL

In [89]:
cursor.execute("""SELECT name, rating 
                                FROM businesses
                                ORDER BY rating DESC
                                LIMIT 5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,name,rating
0,Pisillo Italian Panini - Chelsea,5.0
1,Macelleria Italian Steakhouse,5.0
2,Casa Mangia,5.0
3,Borgatti's Ravioli & Egg Noodles,5.0
4,Pasta Shop,5.0


In [90]:
cursor.execute("""SELECT name, rating 
                                FROM businesses
                                ORDER BY rating
                                LIMIT 5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,name,rating
0,7 Old Fulton Restaurant,2.5
1,Giovanni,2.5
2,Porta,3.0
3,Arte Cafe,3.0
4,Donato's Restaurant,3.0


In [91]:
cursor.execute("""SELECT name, rating
                                FROM  businesses
                                WHERE LENGTH(price) = 1
                                LIMIT 10""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,name,rating
0,Sac's Place Of Jackson Heights,4.5
1,Savino's Quality Pasta,4.5
2,Sicily's Best Pizzeria,4.5
3,Famous Calabria,4.0
4,Lamarca Cheese Shop,4.5
5,Arancini Bros,4.0
6,Leo's Latticini Mama's,4.5
7,Borgatti's Ravioli & Egg Noodles,5.0
8,Raffetto's,4.5
9,Louie's Pizza,4.5


In [92]:
cursor.execute("""SELECT COUNT(business_id)
                      FROM businesses
                      WHERE rating > 4.5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,COUNT(business_id)
0,24


In [93]:

cursor.execute("""SELECT COUNT(business_id)
                      FROM businesses
                      WHERE rating < 3""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,COUNT(business_id)
0,2


In [94]:
cursor.execute("""SELECT COUNT(business_id)
                      FROM businesses
                      WHERE rating < 4""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,COUNT(business_id)
0,220


In [95]:
cursor.execute("""SELECT COUNT(business_id)
                      FROM businesses
                      WHERE rating < 4.5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,COUNT(business_id)
0,721


In [96]:
cursor.execute("""SELECT b.name, avg(r.rating)
                      FROM businesses b
                      JOIN reviews2 r
                      ON r.business_id = b.business_id
                      GROUP BY r.business_id
                      """)
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

,name,avg(r.rating)
0,Palermo Restaurant & Bar,3.666667
1,da Umberto,4.666667
2,Sac's Place Of Jackson Heights,3.333333
3,Trattoria Italienne,5.000000
4,Novita Restaurant,4.333333
5,Ristoro Del Cinghiale,3.666667
6,Cecconi's Dumbo,3.333333
7,Babbo,2.666667
8,The Leopard at des Artistes,2.666667
9,Lex Restaurant,3.333333


In [98]:
cursor.close()
cnx.close()